<a href="https://colab.research.google.com/github/joaosouzaprog-pixel/guia-orcamento-503020/blob/main/Assistente_Voz_Whisper_gTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
# 1. INSTALAÇÃO E IMPORTAÇÃO (Executar primeiro)
# ======================================================
!pip install git+https://github.com/openai/whisper.git gTTS -q

import whisper
from gtts import gTTS
from IPython.display import display, Javascript, Audio
from google.colab import output
import base64

# ======================================================
# 2. FUNÇÃO DE GRAVAÇÃO ROBUSTA (JAVASCRIPT)
# ======================================================
def gravar_audio(nome_arquivo='input_audio.wav'):
    js = Javascript('''
        async function recordAudio() {
            const div = document.createElement('div');
            const button = document.createElement('button');
            const status = document.createElement('span');

            button.textContent = '🎤 Iniciar Gravação';
            button.style.cssText = 'padding: 15px; border-radius: 10px; background: #4285F4; color: white; cursor: pointer; border: none; font-weight: bold;';
            status.style.cssText = 'margin-left: 15px; font-family: sans-serif;';

            div.appendChild(button);
            div.appendChild(status);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            const recorder = new MediaRecorder(stream);
            const chunks = [];

            recorder.ondataavailable = (e) => chunks.push(e.data);
            recorder.onstop = async () => {
                const blob = new Blob(chunks);
                const reader = new FileReader();
                reader.readAsDataURL(blob);
                reader.onloadend = () => {
                    window.callback(reader.result.split(',')[1]);
                };
            };

            button.onclick = () => {
                if (recorder.state === 'inactive') {
                    recorder.start();
                    button.textContent = '⏹️ Parar Gravação';
                    button.style.background = '#EA4335';
                    status.textContent = 'Ouvindo...';
                } else {
                    recorder.stop();
                    button.textContent = '✅ Gravado!';
                    button.style.background = '#34A853';
                    status.textContent = 'Enviando para o Whisper...';
                }
            };

            return new Promise((resolve) => {
                window.callback = resolve;
            });
        }
    ''')

    display(js)
    data = output.eval_js('recordAudio()')
    with open(nome_arquivo, "wb") as f:
        f.write(base64.b64decode(data))
    return nome_arquivo

# ======================================================
# 3. PROCESSAMENTO E ASSISTENTE
# ======================================================

# Carregando o modelo fora da função para não repetir o download
print("Carregando modelo Whisper 'small' (mais preciso)...")
model = whisper.load_model("small")

def executar_assistente():
    # 1. Grava
    audio_file = gravar_audio()

    # 2. Transcreve com parâmetros restritos para PT
    print("\n[Whisper] Analisando áudio...")
    options = dict(language='pt', beam_size=5, best_of=5, fp16=False)
    result = model.transcribe(audio_file, **options)

    texto_final = result["text"].strip()
    print(f"-> Você disse: \"{texto_final}\"")

    # 3. Lógica de Negócio (Escritório)
    texto_l = texto_final.lower()

    if any(x in texto_l for x in ["projeto", "tarefa", "concluído", "finalizado"]):
        resposta = "Entendido! Vou marcar essa tarefa como concluída no sistema."
    elif any(x in texto_l for x in ["reunião", "ata", "anote"]):
        resposta = "Certo, estou registrando esses pontos na ata da reunião."
    elif "ajuda" in texto_l:
        resposta = "Claro! Posso registrar tarefas ou transcrever o que você disser."
    else:
        resposta = "Áudio recebido. Não identifiquei um comando específico, mas a transcrição foi salva."

    # 4. Resposta em Áudio (gTTS)
    print(f"[Assistente] {resposta}")
    tts = gTTS(text=resposta, lang='pt', tld='com.br')
    tts.save("resposta.mp3")

    display(Audio("resposta.mp3", autoplay=True))

# Iniciar o processo
executar_assistente()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Carregando modelo Whisper 'small' (mais preciso)...


<IPython.core.display.Javascript object>